Reference https://blog.video.ibm.com/streaming-video-tips/keyframes-interframe-video-compression/

Code for perforing system sampling of video frames. As of now systematic interval has been set to 5, the interval has to be decided by calculating the mean of key frame interval values from some prominent videos in datasets

In [4]:
!ffmpeg -i '/content/01_original.mp4' -vf showinfo '/content/output.txt'

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Extracting keyfrme using ffmpeg

In [ ]:
!ffmpeg -skip_frame nokey -i '/content/01_original.mp4' -vsync 0 -r 30 -f image2 thumbnails-%02d.jpeg

Reference https://stackoverflow.com/questions/18085458/checking-keyframe-interval

In [14]:
!ffprobe -loglevel error -select_streams v:0 -show_entries packet=pts_time,flags -of csv=print_section=0 '/content/alimi exercise1.mp4' | awk -F',' '/K/ {print $1}'

0.000000
10.000000


Reference https://stackoverflow.com/questions/18085458/checking-keyframe-interval

In [ ]:
!ffmpeg -i '/content/01_original.mp4' -f ffmetadata '/content/output.txt'

In [12]:
!probtime=10 && ffprobe -hide_banner -of compact=p=0:nk=1 -select_streams v:0 -count_frames -show_entries stream=nb_read_frames -skip_frame nokey -v 0 -read_intervals "%+$probtime" -i '/content/alas walk1.AVI' | awk -v probtime="$probtime" '{print probtime/$1}'

0.042735


In [13]:
!probtime=10 && ffprobe -hide_banner -of compact=p=0:nk=1 -select_streams v:0 -count_frames -show_entries stream=nb_read_frames -skip_frame nokey -v 0 -read_intervals "%+$probtime" -i '/content/alimi exercise1.mp4' | awk -v probtime="$probtime" '{print probtime/$1}'

10


Systematic sampling Reference https://www.geeksforgeeks.org/systematic-sampling-in-pandas/

In [1]:
import cv2
import os
out_dir_path='/content/frames'
if not os.path.exists(out_dir_path):
 os.mkdir('/content/frames')
vidcap = cv2.VideoCapture('/content/alas walk1.AVI')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    video_length = round(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    video_fps = round(vidcap.get(cv2.CAP_PROP_FPS),2)
    video_time = round((video_length /video_fps),2)
    video_resolution = str(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)) + "/" + str(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT ))
    if hasFrames:
        cv2.imwrite("/content/frames/image"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 0.5 #//it will capture image in each 0.5 second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [56]:
from os import listdir
from os.path import isfile, join
import numpy
import cv2
import natsort
from skimage.metrics import structural_similarity as ssim
mypath='/content/frames'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
onlyfiles=natsort.natsorted(onlyfiles)
images = numpy.empty(len(onlyfiles), dtype=object)
ssim_list=[]
for n in range(0, len(onlyfiles)):
  images[n] = cv2.imread( join(mypath,onlyfiles[n]) )
print(len(images))
print(images.shape)
print(type(images))
print(images[0])

16
(16,)
<class 'numpy.ndarray'>
[[[250 241 232]
  [250 241 232]
  [250 241 232]
  ...
  [253 242 238]
  [253 242 238]
  [252 241 237]]

 [[250 241 232]
  [250 241 232]
  [250 241 232]
  ...
  [254 243 239]
  [254 243 239]
  [253 242 238]]

 [[250 241 232]
  [250 241 232]
  [250 241 232]
  ...
  [254 245 241]
  [253 244 240]
  [253 244 240]]

 ...

 [[121 126 125]
  [123 128 127]
  [126 131 130]
  ...
  [142 140 139]
  [143 141 140]
  [139 137 136]]

 [[119 124 123]
  [122 127 126]
  [125 130 129]
  ...
  [143 141 140]
  [148 146 145]
  [149 147 146]]

 [[115 120 119]
  [119 124 123]
  [121 126 125]
  ...
  [139 137 136]
  [141 139 138]
  [142 140 139]]]


Code to perform systematic sampling

In [60]:
#sampling_array=images[0]
#print(sampling_array,shape)
sampling_interval=5
sampling_array = numpy.empty(sampling_interval, dtype=object)
i=0
j=0
while(i<=len(images)):
  sampling_array[j]=images[i]
  i=i+sampling_interval
  j=j+1
print(len(sampling_array))



5


In [ ]:
from PIL import Image
from numpy import *
out_dir_path='/content/random sample'
if not os.path.exists(out_dir_path):
 os.mkdir('/content/random_sample')
for i in range(0,5):
  im = array(random_list[i])
  image=Image.fromarray(im)
  image.save("/content/random_sample/rand_Image"+str(i)+".jpg")

In [66]:
from PIL import Image
from numpy import *
out_dir_path='/content/systematic_sample'
if not os.path.exists(out_dir_path):
 os.mkdir('/content/systematic_sample')
for i in range(0,sampling_interval):
  im = array(sampling_array[i])
  print(type(im))
  image = Image.fromarray(im)
  image.save("/content/systematic_sample/sys_Image"+str(i)+".jpg")

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


TypeError: ignored

In [67]:
!pip install PhotoHash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for PhotoHash: filename=Photohash-0.4.1-py3-none-any.whl size=5008 sha256=1a232648699569b93419c45759d72879c1305ee28f0199474ff614ad714345e7
  Stored in directory: /root/.cache/pip/wheels/9d/f5/06/2ae7abda95ec47b571ec47ecbf348fa24a375d6af17e6dbdd5
Successfully built PhotoHash


In [70]:
import photohash
import glob
for file in glob.glob('/content/systematic_sample/*.jpg'):
  hash = photohash.average_hash(file)
  print(hash)
for i in range(0,4):
  distance = photohash.distance('/content/systematic_sample/sys_Image'+str(i)+'.jpg', '/content/systematic_sample/sys_Image'+str(i+1)+'.jpg')
  print(distance)

c7c7c600007eff7f
c7c7c600007edf5f
c7c7c600007efd7e
c7c7c600007eff7f
2
4
2


FileNotFoundError: ignored

In [ ]:
import photohash
similar = photohash.is_look_alike('/content/random_sample/rand_Image3.jpg', '/content/random_sample/rand_Image4.jpg', tolerance=0)
print(similar)	
		
		


False
